## Lecture: The (Py)Tesseract Library

In [ ]:
import PIL
from PIL import Image
from PIL import ImageChops
import pytesseract

import string
from pathlib import Path
from IPython.display import display

In [ ]:
# OCR clean image

text_image = Image.open("/app/readonly/text.png")
display(text_image)

text = pytesseract.image_to_string(text_image)
print(text)

In [ ]:
# greyscale

noisy_img = Image.open("/app/readonly/Noisy_OCR.PNG")
display(noisy_img)
text = pytesseract.image_to_string(noisy_img)
print(text)

img = noisy_img.convert('L')
display(img)
text = pytesseract.image_to_string(img)
print(text)

In [ ]:
# binarization - thresholding

noisy_img = Image.open("/app/readonly/Noisy_OCR.PNG")
img = noisy_img.convert('1')
display(img)

In [ ]:
# binarization with custom threshold

def binarize(image_to_transform, threshold):
    return image_to_transform.convert('L').point(lambda x: 255 if x > threshold else 0, mode='1')

for thresh in range(64, 256, 64):
    print("Trying with threshold " + str(thresh))
    black_and_white = binarize(noisy_img)

    display(black_and_white)
    print(pytesseract.image_to_string(black_and_white))

## Tesseract and Photographs

In [ ]:
# OCR on a picture

image = Image.open('/app/readonly/storefront.jpg')
display(image)
pytesseract.image_to_string(image).strip()

In [ ]:
# Crop + OCR

title_image = image.crop( (315, 170, 700, 270) )
display(title_image)
print(pytesseract.image_to_string(title_image).strip())

little_sign = image.crop( (900, 420, 940, 445) )
display(little_sign)
pytesseract.image_to_string(little_sign).strip()

In [ ]:
# Resize options

new_size = (little_sign.width * 8, little_sign.height * 8)
options=[Image.NEAREST, Image.BOX, Image.BILINEAR, Image.HAMMING, Image.BICUBIC, Image.LANCZOS]
for option in options:
    print(option)
    display(little_sign.resize( new_size, option))

In [ ]:
# binarize + OCR - Use a dictionnary to autodetect if text found has a meaning

eng_words = set(Path("/app/readonly/words_alpha.txt").read_text().split("\n"))
for threshold in range(140, 160, 4):
    black_white_image = binarize(bigger_sign, threshold)
    #black_white_image = ImageChops.invert(black_white_image)
    text = pytesseract.image_to_string(black_white_image).strip()
    
    # Normalize text : lowercase + remove non alphabetical characters, like ([%$])
    clean_text = "".join(
        character
        for character in text.lower()
        if character in string.ascii_lowercase
    )

    if clean_text in eng_words:
        display(black_white_image)
        print(threshold, clean_text)

## Jupyter Widgets (Optional)

In [ ]:
# Add sliders in the notebook to easily change a parameter of a function
# The decorator will take a set of parameters which are identical to the function to be called.

# If you would like to explore more you can read about what is available here: 
# https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

from ipywidgets import interact

image = Image.open('/app/readonly/storefront.jpg')

@interact(left=100, top=100, right=200, bottom=200)
def draw_border(left, top, right, bottom):
    img = image.copy()
    drawing_object = ImageDraw.Draw(img)
    drawing_object.rectangle((left,top,right,bottom), fill = None, outline ='red')
    display(img)